In [8]:
import pandas as pd
import numpy as np

import seaborn as sns ;sns.set(rc={'figure.figsize':(20,10)})

import matplotlib.pyplot as plt
plt.rcParams ['figure.figsize']  = 25, 10
plt.rcParams ['axes.labelsize']  = 14
plt.rcParams ['xtick.labelsize'] = 12
plt.rcParams ['ytick.labelsize'] = 12
plt.rcParams ['text.color']      = 'black'
plt.rcParams ['axes.labelcolor'] = 'black'
plt.rcParams ['xtick.color']     = 'black'
plt.rcParams ['ytick.color']     = 'black'
plt.style.use('fivethirtyeight')


import warnings
warnings.filterwarnings("ignore")


pd.options.display.float_format = '{:,.5f}'.format
pd.options.display.max_rows = 150
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 0


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Explore datasets

In [131]:
kidney_dataset = pd.read_csv('../data/kidney_disease/kidney_disease.csv')

In [132]:
kidney_dataset.sample(5)

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,bu,sc,sod,pot,hemo,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
4,4,51.00000,80.00000,1.01000,2.00000,0.00000,normal,normal,notpresent,notpresent,106.00000,26.00000,1.40000,nan,nan,11.60000,35,7300,4.6,no,no,no,good,no,no,ckd
286,286,71.00000,70.00000,1.02000,0.00000,0.00000,normal,normal,notpresent,notpresent,81.00000,18.00000,0.80000,145.00000,5.00000,14.70000,44,9800,6.0,no,no,no,good,no,no,notckd
360,360,35.00000,60.00000,1.02500,0.00000,0.00000,normal,normal,notpresent,notpresent,105.00000,39.00000,0.50000,135.00000,3.90000,14.70000,43,5800,6.2,no,no,no,good,no,no,notckd
276,276,20.00000,60.00000,1.02500,0.00000,0.00000,normal,normal,notpresent,notpresent,nan,nan,nan,137.00000,4.70000,14.00000,41,4500,5.5,no,no,no,good,no,no,notckd
190,190,6.00000,60.00000,1.01000,4.00000,0.00000,abnormal,abnormal,notpresent,present,94.00000,67.00000,1.00000,135.00000,4.90000,9.90000,30,16700,4.8,no,no,no,poor,no,no,ckd


In [133]:
kidney_dataset.describe()

,id,age,bp,sg,al,su,bgr,bu,sc,sod,pot,hemo
count,400.00000,391.00000,388.00000,353.00000,354.00000,351.00000,356.00000,381.00000,383.00000,313.00000,312.00000,348.00000
mean,199.50000,51.48338,76.46907,1.01741,1.01695,0.45014,148.03652,57.42572,3.07245,137.52875,4.62724,12.52644
std,115.61430,17.16971,13.68364,0.00572,1.35268,1.09919,79.28171,50.50301,5.74113,10.40875,3.19390,2.91259
min,0.00000,2.00000,50.00000,1.00500,0.00000,0.00000,22.00000,1.50000,0.40000,4.50000,2.50000,3.10000
25%,99.75000,42.00000,70.00000,1.01000,0.00000,0.00000,99.00000,27.00000,0.90000,135.00000,3.80000,10.30000
50%,199.50000,55.00000,80.00000,1.02000,0.00000,0.00000,121.00000,42.00000,1.30000,138.00000,4.40000,12.65000
75%,299.25000,64.50000,80.00000,1.02000,2.00000,0.00000,163.00000,66.00000,2.80000,142.00000,4.90000,15.00000
max,399.00000,90.00000,180.00000,1.02500,5.00000,5.00000,490.00000,391.00000,76.00000,163.00000,47.00000,17.80000


In [136]:
def clean_noisy_data(dataset,classes = 2):
    
    if 'id' in dataset.columns :
        dataset = dataset.drop(columns = ['id']) # Drop id column as it's not relevent for predictions

    #Changing numerical data into float types except for the output variable
    string_columns = []
    for c in dataset.columns : 
        try : 
            dataset[c] = dataset[c].astype(float)
        except ValueError :
            string_columns.append(c)
            
    for c in string_columns :
        dataset[c] = dataset[c].str.replace('\t','')
        dataset[c] = dataset[c].replace('?',np.nan)
    
    #Ordinal encoding of the output variable 
    output_column = dataset.columns[-1]
    outputs = dataset[output_column].unique()
    assert len(outputs) == classes, 'Error the number of output classes should be the same as the one in the dataset'
    dataset[output_column] = 1*(dataset[output_column] == outputs[0]) #return an ordinal encoding of the output variable
    
    return dataset

In [138]:
clean_noisy_data(kidney_dataset).sample(5)

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,bu,sc,sod,pot,hemo,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
258,42.00000,80.00000,1.02000,0.00000,0.00000,normal,normal,notpresent,notpresent,98.00000,20.00000,0.50000,140.00000,3.50000,13.90000,44,8400,5.5,no,no,no,good,no,no,0
288,56.00000,70.00000,1.02500,0.00000,0.00000,normal,normal,notpresent,notpresent,70.00000,46.00000,1.20000,135.00000,4.90000,15.90000,50,11000,5.1,NaN,NaN,NaN,good,no,no,0
124,65.00000,100.00000,1.01500,0.00000,0.00000,NaN,normal,notpresent,notpresent,90.00000,98.00000,2.50000,nan,nan,9.10000,28,5500,3.6,yes,no,no,good,no,no,1
177,65.00000,80.00000,1.01500,2.00000,1.00000,normal,normal,present,notpresent,215.00000,133.00000,2.50000,nan,nan,13.20000,41,NaN,NaN,no,yes,no,good,no,no,1
341,63.00000,70.00000,1.02500,0.00000,0.00000,normal,normal,notpresent,notpresent,130.00000,37.00000,0.90000,150.00000,5.00000,13.40000,41,7300,4.7,no,no,no,good,no,no,0


In [6]:
bank_dataset = pd.read_csv('../data/banknote/data_banknote_authentication.txt',sep=',')

In [7]:
bank_dataset.sample(5)

,variance,skewness,curtosis,entropy,class
1341,-2.26250,-0.09934,2.81270,0.48662,1
54,4.92640,5.49600,-2.47740,-0.50648,0
693,1.54780,9.18140,-1.63260,-1.73750,0
752,0.38478,6.59890,-0.33360,-0.56466,0
210,4.28990,9.18140,-4.60670,-4.32630,0


In [139]:
clean_noisy_data(bank_dataset)

,variance,skewness,curtosis,entropy,class
0,3.62160,8.66610,-2.80730,-0.44699,1
1,4.54590,8.16740,-2.45860,-1.46210,1
2,3.86600,-2.63830,1.92420,0.10645,1
3,3.45660,9.52280,-4.01120,-3.59440,1
4,0.32924,-4.45520,4.57180,-0.98880,1
...,...,...,...,...,...
1367,0.40614,1.34920,-1.45010,-0.55949,0
1368,-1.38870,-4.87730,6.47740,0.34179,0
1369,-3.75030,-13.45860,17.59320,-2.77710,0
1370,-3.56370,-8.38270,12.39300,-1.28230,0
